In [1]:
pip install pinecone pypdf2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata


In [3]:
import tqdm
import warnings
warnings.filterwarnings('ignore')
import google.generativeai as genai
import time
from pinecone import Pinecone, ServerlessSpec
from PyPDF2 import PdfReader
import re
_RE_COMBINE_WHITESPACE = re.compile(r"(?a:\s+)")
_RE_STRIP_WHITESPACE = re.compile(r"(?a:^\s+|\s+$)")
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

In [4]:

index_name = 'mlchatbotqa'
# pc.create_index(
#     name=index_name,
#     dimension=768 ,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [5]:
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [6]:
reader = PdfReader('/content/Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf')
number_of_pages = len(reader.pages)

# for i in tqdm.tqdm(range(0,number_of_pages)):
#     text_in_page = ''
#     page = reader.pages[i]
#     text_in_page = page.extract_text()
#     text_in_page = text_in_page.replace('\n',' ')
#     text_in_page = _RE_COMBINE_WHITESPACE.sub(" ", text_in_page)
#     text_in_page = _RE_STRIP_WHITESPACE.sub("", text_in_page)
#     if(len(text_in_page)>9000):
#         text_in_page = text_in_page[:9000]
#     gemini_embedding_result= genai.embed_content(
#         model="models/text-embedding-004",
#         content= text_in_page,
#         task_type="retrieval_document",
#         title="Embedding of single string")
#     index.upsert(
#         vectors=[
#             {
#             "id": 'page-' + str(i) + '-'+ str(i+1),
#             "values": gemini_embedding_result['embedding'] ,
#             "metadata": {"text":text_in_page}
#             }
#         ]
#     )

from math import ceil

for i in tqdm.tqdm(range(14, number_of_pages)):
    text_in_page = ''
    page = reader.pages[i]
    text_in_page = page.extract_text()
    text_in_page = text_in_page.replace('\n', ' ')
    text_in_page = _RE_COMBINE_WHITESPACE.sub(" ", text_in_page)
    text_in_page = _RE_STRIP_WHITESPACE.sub("", text_in_page)

    # Splitting the text of any page into 5 parts for better result
    num_parts = 1
    chunk_size = ceil(len(text_in_page) / num_parts)

    for j in range(num_parts):
        start_idx = j * chunk_size
        end_idx = min((j + 1) * chunk_size, len(text_in_page))
        chunk = text_in_page[start_idx:end_idx]


        if len(chunk) > 0:
            gemini_embedding_result = genai.embed_content(
                model="models/text-embedding-004",
                content=chunk,
                task_type="retrieval_document",
                title=f"Embedding of page {i+1}, part {j+1}"
            )
        index.upsert(
                vectors=[
                    {
                        "id": f'page-{i+1}-part-{j+1}',
                        "values": gemini_embedding_result['embedding'],
                        "metadata": {"text": chunk}
                    }
                ]
            )


100%|██████████| 378/378 [12:36<00:00,  2.00s/it]


In [7]:
question = "who is the president of America"

In [8]:
vectordb_top_matches = ''
gemini_embedding_question= genai.embed_content(
        model="models/text-embedding-004",
        content=  question,
        task_type="retrieval_document",
        title="Embedding of single string")


query_result = index.query(
            vector = gemini_embedding_question['embedding'],
            top_k = 5,
            include_metadata=True)
for result_pinecone in query_result['matches']:
    vectordb_top_matches += result_pinecone['metadata']['text']

generation_config = {
  "temperature": 0.4,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

gen_model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

In [9]:
print(query_result)

{'matches': [{'id': 'page-163-part-1',
              'metadata': {'text': 'Out[23]: Alejandro Toledo 39 Alvaro Uribe '
                                   '35 Amelie Mauresmo 21 Andre Agassi '
                                   '36Angelina Jolie 20 Arnold Schwarzenegger '
                                   '42Atal Bihari Vajpayee 24 Bill Clinton '
                                   '29Carlos Menem 21 Colin Powell 236David '
                                   'Beckham 31 Donald Rumsfeld 121George W '
                                   'Bush 530 George Robertson 22Gerhard '
                                   'Schroeder 109 Gloria Macapagal Arroyo '
                                   '44Gray Davis 26 Guillermo Coria 30Hamid '
                                   'Karzai 22 Hans Blix 39Hugo Chavez 71 Igor '
                                   'Ivanov 20[...] [...]Laura Bush 41 Lindsay '
                                   'Davenport 22Lleyton Hewitt 41 Luiz Inacio '
                            

In [10]:
input_question_prompt= f"""please act as a expert interpreater and answer the following questions {question} from the input text provided.
If the answer is not in the input text please respond back with “Sorry, I didn’t understand your question. Do you want to connect with a live agent?” """


model_response = gen_model.generate_content([input_question_prompt,vectordb_top_matches])

result = model_response.text
print(result)

Sorry, I didn’t understand your question. Do you want to connect with a live agent?



In [11]:
def question_answer():
  question = input("Please enter your question: ")
  vectordb_top_matches = ''
  gemini_embedding_question= genai.embed_content(
          model="models/text-embedding-004",
          content=  question,
          task_type="retrieval_document",
          title="Embedding of single string")

  query_result = index.query(
              vector = gemini_embedding_question['embedding'],
              top_k = 5,
              include_metadata=True)
  for result_pinecone in query_result['matches']:
      vectordb_top_matches += result_pinecone['metadata']['text']

  generation_config = {
    "temperature": 0.4,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
  }

  gen_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
  )

  input_question_prompt= f"""please act as a expert interpreater and answer the following questions {question} from the input text provided.
  If the answer is not in the input text please respond back with “Sorry, I didn’t understand your question. Do you want to connect with a live agent?” """


  model_response = gen_model.generate_content([input_question_prompt,vectordb_top_matches])

  result = model_response.text
  return result


# Checking with Different Questions

In [12]:
question_answer()

Please enter your question: What is the classification


'The provided text focuses on **text classification**.  Specifically, it uses the example of classifying movie reviews as positive or negative, and mentions other examples like classifying emails as spam or not spam, and customer service messages as complaints or inquiries.\n'

In [16]:
question_answer()

Please enter your question: What are the Assumptions in Linear Regression


'Based on the provided text, a key assumption of linear regression is that the target variable (y) is a linear combination of the features (x).  The text explicitly states this as a "strong (and somewhat unrealistic) assumption".  The text also implies that this assumption might be less problematic with datasets containing many features.\n'

In [13]:
pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.3 MB/s eta 0:00:00


In [14]:
import gradio as gr
interface = gr.Interface(
    fn=question_answer,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your question here...", label="Question"),
    ],
    outputs=gr.Textbox(label="Answer"),
    title="Question-Answer Chatbot on Machine Learning Concepts",
    description="Ask a question. The chatbot will extract the answer from the given context."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f19763063c33cf3df3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
